### Customer Churn model prediction [which customers are likely to leave or unsubscribe the bank] using Artificial Neural Network

#### Importing the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

#### Importing the dataset

In [2]:
dataset = pd.read_csv('../Dataset/Churn_Modelling.csv')
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


## Part 1 - Data Preprocessing

#### Splitting the dataset into independent and dependent variables

In [3]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

#### Encoding categorical data

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])     # LabelEncoding Gender column

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')           # OneHotEncoder Geography column
X = np.array(ct.fit_transform(X))

# Taking care of dummy variable trap for the Geography
X = X[:, 1:]

#### Splitting the dataset into the Training set and Test set 

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

#### Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

#### Importing the Keras libraries and packages

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

#### Initialization the ANN

In [8]:
classifier = Sequential()

#### Adding the input layer and the first hidden layer

In [9]:
# Units => Output_dimention [ input_dim + output_dim / 2 ] || [ (11 + 1) / 2]
# kernel_initializer => Distribution of weights (Uniformly randomly initializing the weights)
# activation => Relu / Rectifier function

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))   # Input layer + First Hidden Layer

#### Adding the second hidden layer and the output layer

In [10]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))       # Second Hidden Layer

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))    # Output layer

#### Compile the ANN

In [14]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 

# Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure to update network weights iterative based in training data as a backpropagation

#### Fitting the ANN to the Training set

In [15]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

# Mini-batch (batch_size = 10)

Epoch 1/100
750/750 [==============================] - 2s 1ms/step - loss: 0.4006 - accuracy: 0.8357
Epoch 2/100
750/750 [==============================] - 1s 2ms/step - loss: 0.4008 - accuracy: 0.8344
Epoch 3/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4006 - accuracy: 0.8351
Epoch 4/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4005 - accuracy: 0.8335
Epoch 5/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4002 - accuracy: 0.8343
Epoch 6/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4005 - accuracy: 0.8353
Epoch 7/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4008 - accuracy: 0.8356
Epoch 8/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4005 - accuracy: 0.8359
Epoch 9/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4006 - accuracy: 0.8335
Epoch 10/100
750/750 [==============================] - 1s 1ms/step - loss: 0.4007 - accura

## Part 3 - Making the predictions and evaluating the model

#### Predicting the Test set results

In [17]:
y_pred_total = classifier.predict(X_test)


79/79 [==============================] - 0s 1ms/step


In [20]:
y_pred = (y_pred_total > 0.5)

# Customers with probability > 50 % will leave the bank

#### Making the Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)       
accuracy_score(y_test, y_pred)


[[1937   54]
 [ 340  169]]


0.8424

##### Accuracy Score of Training Set => 0.8348
##### Accuracy Score of Test Set => 0.8424